# Deep Learning CNN model to recognize faces

#### Specifying training path

In [1]:
trainpath = 'Face Images/Final Training Images'

#### Image pre-processing

In [5]:
from keras.preprocessing.image import ImageDataGenerator

In [7]:
# Defining pre-processing transformations on raw images of training data
# These hyper parameters helps to generate slightly twisted versions
# of the original image, which leads to a better model, since it learns
# on the good and bad mix of images
train_datagen = ImageDataGenerator(
    shear_range=0.1,
    zoom_range=0.1,
    horizontal_flip=True
)

In [8]:
# Defining pre-processing transformations on raw images of testing data
# No transformations are done on the testing images
test_datagen = ImageDataGenerator()

In [9]:
# Generate training data
training_set = train_datagen.flow_from_directory(
    trainpath,
    target_size=(64,64),
    batch_size=32,
    class_mode='categorical'
)

Found 244 images belonging to 16 classes.


In [10]:
# Generate testing data
testing_set = test_datagen.flow_from_directory(
    trainpath,
    target_size=(64,64),
    batch_size=32,
    class_mode='categorical'
)

Found 244 images belonging to 16 classes.


In [12]:
# Printing class labels for each face
testing_set.class_indices

{'face1': 0,
 'face10': 1,
 'face11': 2,
 'face12': 3,
 'face13': 4,
 'face14': 5,
 'face15': 6,
 'face16': 7,
 'face2': 8,
 'face3': 9,
 'face4': 10,
 'face5': 11,
 'face6': 12,
 'face7': 13,
 'face8': 14,
 'face9': 15}

#### Creating lookup table for all faces

In [13]:
# class_indices have the numeric tag for each face
train_classes = training_set.class_indices

# storing the faces and the numeric tag for future reference
resultMap = {}
for faceValue, faceName in zip(train_classes.values(),train_classes.keys()):
    resultMap[faceValue] = faceName

# saving the face map for future reference
import pickle
with open('resultMap.pkl','wb') as file:
    pickle.dump(resultMap, file)
file.close()

print("Mapping of Face and its ID:",resultMap)

# the number of neurons for the output layer is equal to number of faces
output_neurons = len(resultMap)
print('\n The number of output neurons:',output_neurons)

Mapping of Face and its ID: {0: 'face1', 1: 'face10', 2: 'face11', 3: 'face12', 4: 'face13', 5: 'face14', 6: 'face15', 7: 'face16', 8: 'face2', 9: 'face3', 10: 'face4', 11: 'face5', 12: 'face6', 13: 'face7', 14: 'face8', 15: 'face9'}

 The number of output neurons: 16


#### Creating the CNN model

In [14]:
from keras.models import Sequential
from keras.layers import Convolution2D, MaxPool2D, Flatten, Dense

# initializing the CNN
classifier = Sequential()

# step 1 : convolution
classifier.add(Convolution2D(32, kernel_size=(5,5), strides=(1,1),input_shape = (64,64,3), activation='relu'))

# step 2 : max pooling
classifier.add(MaxPool2D(pool_size=(2,2)))

# additional layer for convolution and max pooling
classifier.add(Convolution2D(64, kernel_size=(5,5), strides=(1,1), activation='relu'))

classifier.add(MaxPool2D(pool_size=(2,2)))

# step 3 : flattening
classifier.add(Flatten())

# step 4 : fully connected neural network
classifier.add(Dense(64, activation='relu'))

classifier.add(Dense(output_neurons, activation='softmax'))

classifier.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d (Conv2D)             (None, 60, 60, 32)        2432      
                                                                 
 max_pooling2d (MaxPooling2D  (None, 30, 30, 32)       0         
 )                                                               
                                                                 
 conv2d_1 (Conv2D)           (None, 26, 26, 64)        51264     
                                                                 
 max_pooling2d_1 (MaxPooling  (None, 13, 13, 64)       0         
 2D)                                                             
                                                                 
 flatten (Flatten)           (None, 10816)             0         
                                                                 
 dense (Dense)               (None, 64)                6

2023-05-11 19:07:23.793773: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  SSE4.1 SSE4.2
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.


In [17]:
# compiling the CNN
classifier.compile(loss='categorical_crossentropy',optimizer='adam',metrics=['accuracy'])

import time
# measuring time taken to train the model
start_time = time.time()

# starting model training
classifier.fit_generator(
    training_set,
    steps_per_epoch=5,
    epochs = 10,
    validation_data=testing_set,
    validation_steps=10
)
end_time = time.time()
print('Training time:',round((end_time-start_time)/60),'minutes')

Epoch 1/10


/var/folders/yz/4zp0zzb50w71fn6jwfwny__c0000gn/T/ipykernel_35268/3671566285.py:9: UserWarning: `Model.fit_generator` is deprecated and will be removed in a future version. Please use `Model.fit`, which supports generators.
  classifier.fit_generator(


5/5 [==============================] - 1s 199ms/step - loss: 2.8231 - accuracy: 0.1125 - val_loss: 2.9560 - val_accuracy: 0.0656
Epoch 2/10
5/5 [==============================] - 0s 77ms/step - loss: 3.0644 - accuracy: 0.1486
Epoch 3/10
5/5 [==============================] - 0s 81ms/step - loss: 2.4417 - accuracy: 0.2375
Epoch 4/10
5/5 [==============================] - 0s 83ms/step - loss: 2.6689 - accuracy: 0.2635
Epoch 5/10
5/5 [==============================] - 0s 74ms/step - loss: 1.8779 - accuracy: 0.4054
Epoch 6/10
5/5 [==============================] - 0s 80ms/step - loss: 1.4216 - accuracy: 0.5541
Epoch 7/10
5/5 [==============================] - 0s 85ms/step - loss: 0.6934 - accuracy: 0.8108
Epoch 8/10
5/5 [==============================] - 0s 86ms/step - loss: 0.5387 - accuracy: 0.8176
Epoch 9/10
5/5 [==============================] - 0s 80ms/step - loss: 0.4152 - accuracy: 0.8875
Epoch 10/10
5/5 [==============================] - 0s 80ms/step - loss: 0.1829 - accuracy: 0.97

#### Testing the CNN classifier on unseen images

In [20]:
# making single predictions
import numpy as np
import keras.utils as image

imgpath = 'Face Images/Final Testing Images/face4/3face4.jpg'
test_img = image.load_img(imgpath,target_size = (64,64))
test_img = image.img_to_array(test_img)

test_img = np.expand_dims(test_img,axis=0)
result = classifier.predict(test_img,verbose=0)
print("Prediction is:",resultMap[np.argmax(result)])

Prediction is: face4
